[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/itmorn/AI.handbook/blob/main/DL/torch/nn/Transformer/TransformerDecoderLayer.ipynb)

# TransformerDecoderLayer
TransformerDecoderLayer 是 PyTorch 中实现 Transformer 解码器的一个模块，它是构成整个 Transformer 解码器的基本单元之一。

在机器翻译等自然语言处理任务中，Transformer 解码器通常用于生成目标语言的句子。它由多个相同的 TransformerDecoderLayer 模块堆叠而成，每个模块都包含了自注意力机制（self-attention）和多头注意力机制（multi-head attention）等组件，以及前馈神经网络（feedforward neural network）。

TransformerDecoderLayer 的主要作用是对编码器的输出进行解码，从而生成目标语言的句子。它使用了自注意力机制和多头注意力机制来处理输入序列和上一层的输出，以捕捉序列中的长程依赖关系和全局语义信息。同时，前馈神经网络可以对这些信息进行进一步的处理和特征提取，以便更好地生成目标语言的句子。


**定义**：  
torch.nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward=2048, dropout=0.1, activation=<function relu>, layer_norm_eps=1e-05, batch_first=False, norm_first=False, device=None, dtype=None)

**参数**:  
- d_model (int) – the number of expected features in the input (required).  输入中特征维度(必需的)。

- nhead (int) – the number of heads in the multiheadattention models (required).  多头注意力模型中的头数(必需的)。

- dim_feedforward (int) – the dimension of the feedforward network model (default=2048).  feedforward模块中Linear的维度

- dropout (float) – the dropout value (default=0.1).  dropout值(默认=0.1)。

- activation (Union[str, Callable[[Tensor], Tensor]]) – the activation function of the intermediate layer, can be a string (“relu” or “gelu”) or a unary callable. Default: relu  中间层的激活函数，可以是一个字符串(" relu "或" gelu ")或一个一元的可调用对象。默认值:relu

- layer_norm_eps (float) – the eps value in layer normalization components (default=1e-5).  层归一化组件中的eps值(默认=1e-5)。

- batch_first (bool) – If True, then the input and output tensors are provided as (batch, seq, feature). Default: False (seq, batch, feature).  如果为True，则输入和输出张量将作为(batch, seq, feature)提供。默认值:False (seq, batch, feature)。

- norm_first (bool) – if True, layer norm is done prior to attention and feedforward operations, respectively. Otherwise it’s done after. Default: False (after).  如果为True，则层norm分别在注意和前馈操作之前完成。否则以后再做。默认值:False(之后)。


# 图解TransformerDecoderLayer
Transformer的结构如下：
<p align="center">
<img src="./imgs/transformer.jpg"
    width="700" /></p>
右边的方块就是TransformerDecoderLayer，另外对TransformerDecoderLayer进行堆叠（N×），可以得到TransformerDecoder。

下面对TransformerDecoderLayer的内部进行详细描述：
<p align="center">
<img src="./imgs/TransformerDecoderLayer.svg"
    width="2000" /></p>

从图中可以看出TransformerDecoderLayer和TransformerEncoderLayer非常像，它们共用了很多结构。  
不同点在于DecoderLayer不仅像EncoderLayer一样做了self-attention，而且又加了一层self-attention，这一层self-attention的q由DecoderLayer来确定，而kv是根据EncoderLayer的输出来计算出来的。

In [21]:
# 单个样本简单举例
import torch
import torch.nn as nn

torch.manual_seed(6688)
d_model = 4

decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=2, dim_feedforward=5, dropout=0.0)

memory = torch.tensor([[[-0.1235, 1.6189, -1.0983, -0.3970]],
                       [[-1.5583, 1.2149, 0.0358, 0.3076]],
                       [[1.4580, 0.3129, -0.5627, -1.2082]]]) # TransformerEncoder的输出L_encoder_out, N, E

# TransformerEncoder的输出L_encoder_out和TransformerDecoder的输入L_decoder_in可以是不同的。

tgt = torch.rand(2, 1, d_model)  # TransformerDecoder的输入L_decoder_in, N, E
out = decoder_layer(tgt, memory)
print("out:\n", out, "\n")  # 可以看到和上图输出的结果是一致的


out:
 tensor([[[ 1.5059,  0.2510, -0.6276, -1.1293]],

        [[-0.5577,  1.1509, -1.3426,  0.7494]]],
       grad_fn=<NativeLayerNormBackward0>) 



# forward函数

**定义**：
forward(tgt, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None)

**参数**：
- tgt (Tensor) – the sequence to the decoder layer (required).  喂给解码器层的序列(必需的)。

- memory (Tensor) – the sequence from the last layer of the encoder (required).  来自编码器最后一层的序列，也叫做memory(必需的)。

- tgt_mask (Optional[Tensor]) – the mask for the tgt sequence (optional).  tgt序列的掩码(可选)

- memory_mask (Optional[Tensor]) – the mask for the memory sequence (optional).  memory序列的掩码(可选)。

- tgt_key_padding_mask (Optional[Tensor]) – the mask for the tgt keys per batch (optional).  每批tgt的key的掩码(可选)。

- memory_key_padding_mask (Optional[Tensor]) – the mask for the memory keys per batch (optional).   每批memory的key的掩码(可选)。